# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [19]:
## !pip install --upgrade scipy==1.1.0

In [20]:
import datetime
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

from scipy.misc import imread, imresize
import datetime 
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

In [21]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam,SGD
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, ZeroPadding3D, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM, GRU

We set the random seed so that the results don't vary drastically.

In [22]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30) 

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [23]:
import pandas as pd
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [24]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,30,2)] #create a list of image numbers you want to use for a particular video
    # considering all the even indices from 0 to 28 with a step of 2 (odd frames from frame 1 to 29 within the video)
    
    x= len(img_idx) #x: number of images used for each video
    y= 80
    z= 80
    # (y,z) is the final size of the input images and 3 is the number of channels RGB
    #Assuming the size of cropped input image is consistent: 80x80
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//(batch_size)  # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output (output: 0,1,2,3,4. Hence 5 possible outputs)
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image_path= source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]
                    image = cv2.imread(image_path, cv2.IMREAD_COLOR).astype(np.float32)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    #Crop the images and resize them. Note that the images are of 2 different shapes 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if image.shape[0]!= image.shape[1]:  ##Checking for square frames 360x360
                        image= image[0:120, 20:140, :]   ##Crops Rectangular frames of 120x160 to 120x120
                    
                    image = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)  #Resizing 360x360 and 120x120 (cropped) images to 80x80

                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if len(t)!= num_batches*batch_size:
            batch_size= len(t) - (num_batches*batch_size)
            batch= num_batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output (output: 0,1,2,3,4. Hence 5 possible outputs)
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image_path= source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]
                    image = cv2.imread(image_path, cv2.IMREAD_COLOR).astype(np.float32)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    #crop the images and resize them. Note that the images are of 2 different shapes 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    if image.shape[0]!= image.shape[1]:  ##Checking for square frames 360x360
                        image= image[0:120, 20:140, :]   ##Crops Rectangular frames to 120x120

                    image = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)  #Resizing 360x360 and 120x120 (cropped) images to 80x80

                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255 #normalise and feed in the image
    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [25]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D`. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

# Conv3D Models

In [26]:
input_shape= (15,80,80,3)

## Model 1

In [27]:
## Model1

# 16,16,32 kernels in 3 consecutive conv3D layers
model1 = Sequential()
# 1st Layer Group
model1.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model1.add(Conv3D(16, kernel_size=(3,3,3),padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model1.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling3D(pool_size=(2,2,2)))

# FC layer group
model1.add(Flatten())
model1.add(Dense(512 , activation='relu'))
model1.add(Dropout(0.5))

# Softmax Layer
model1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [28]:
optimiser = Adam() #write your optimizer
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 80, 80, 16)    1312      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 40, 40, 16)     6928      
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 40, 40, 16)     64        
_________________________________________________________________
activation_2 (Activation)    (None, 7, 40, 40, 16)     0         
__________

## Model 2

In [29]:
# Model 2
# 16,32,32 kernels in 3 consecutive conv3D layers
model2 = Sequential()
# 1st Layer Group
model2.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model2.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model2.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling3D(pool_size=(2,2,2)))

# FC layer group
model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
# Softmax Layer
model2.add(Dense(5, activation='softmax'))

In [30]:
optimiser = Adam() #write your optimizer
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 15, 80, 80, 16)    1312      
_________________________________________________________________
batch_normalization_4 (Batch (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_4 (Activation)    (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 7, 40, 40, 32)     13856     
_________________________________________________________________
batch_normalization_5 (Batch (None, 7, 40, 40, 32)     128       
_________________________________________________________________
activation_5 (Activation)    (None, 7, 40, 40, 32)     0         
__________

## Model 3

In [31]:
# Model 3
# 16,32,64 kernels in 3 consecutive conv3D layers
model3 = Sequential()
# 1st Layer Group
model3.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model3.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model3.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(MaxPooling3D(pool_size=(2,2,2)))

# FC layer group
model3.add(Flatten())
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.5))
# Softmax Layer
model3.add(Dense(5, activation='softmax'))

In [32]:
optimiser = Adam() #write your optimizer
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 15, 80, 80, 16)    1312      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_7 (Activation)    (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 7, 40, 40, 32)     13856     
_________________________________________________________________
batch_normalization_8 (Batch (None, 7, 40, 40, 32)     128       
_________________________________________________________________
activation_8 (Activation)    (None, 7, 40, 40, 32)     0         
__________

## Model 4

In [33]:
# Model 4
# 16,64,64 kernels in 3 consecutive conv3D layers
model4 = Sequential()
# 1st Layer Group
model4.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model4.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model4.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling3D(pool_size=(2,2,2)))

# FC layer group
model4.add(Flatten())
model4.add(Dense(512, activation='relu'))
model4.add(Dropout(0.5))
# Softmax Layer
model4.add(Dense(5, activation='softmax'))

In [34]:
optimiser = Adam() #write your optimizer
model4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 15, 80, 80, 16)    1312      
_________________________________________________________________
batch_normalization_10 (Batc (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_10 (Activation)   (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 7, 40, 40, 64)     27712     
_________________________________________________________________
batch_normalization_11 (Batc (None, 7, 40, 40, 64)     256       
_________________________________________________________________
activation_11 (Activation)   (None, 7, 40, 40, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

## Model 5

In [35]:
# Model 5
# 16,32,32 kernels in 3 consecutive conv3D layers
model5 = Sequential()
# 1st Layer Group
model5.add(Conv3D(16, kernel_size=(2,2,2), input_shape=input_shape,padding='same'))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model5.add(Conv3D(32, kernel_size=(2,2,2),padding='same'))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model5.add(Conv3D(32, kernel_size=(2,2,2),padding='same'))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling3D(pool_size=(2,2,2)))

# FC layer group
model5.add(Flatten())
model5.add(Dense(512,activation='relu'))
model5.add(Dropout(0.5))

model5.add(Dense(128,activation='relu'))
model5.add(Dropout(0.25))

# Softmax Layer
model5.add(Dense(5, activation='softmax'))

In [36]:
optimiser = Adam() #write your optimizer
model5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 15, 80, 80, 16)    400       
_________________________________________________________________
batch_normalization_13 (Batc (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_13 (Activation)   (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 7, 40, 40, 32)     4128      
_________________________________________________________________
batch_normalization_14 (Batc (None, 7, 40, 40, 32)     128       
_________________________________________________________________
activation_14 (Activation)   (None, 7, 40, 40, 32)     0         
__________

## Model 6

In [37]:
# Model 6
# 16,32,32 kernels in 3 consecutive conv3D layers
model6 = Sequential()
# 1st Layer Group
model6.add(Conv3D(16, kernel_size=(5,5,5), input_shape=input_shape,padding='same'))
model6.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model6.add(Conv3D(32, kernel_size=(5,5,5),padding='same'))
model6.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model6.add(Conv3D(32, kernel_size=(5,5,5),padding='same'))
model6.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(MaxPooling3D(pool_size=(2,2,2)))

#FC layer group
model6.add(Flatten())
model6.add(Dense(512,activation='relu'))
model6.add(Dropout(0.5))

model6.add(Dense(128,activation='relu'))
model6.add(Dropout(0.25))

# Softmax Layer
model6.add(Dense(5, activation='softmax'))

In [38]:
optimiser = Adam() #write your optimizer
model6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model6.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 15, 80, 80, 16)    6016      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_16 (Activation)   (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 7, 40, 40, 32)     64032     
_________________________________________________________________
batch_normalization_17 (Batc (None, 7, 40, 40, 32)     128       
_________________________________________________________________
activation_17 (Activation)   (None, 7, 40, 40, 32)     0         
__________

## Model 7

In [39]:
# Model 7
# 32,64,64 kernels in 3 consecutive conv3D layers
model7 = Sequential()
# 1st Layer Group
model7.add(Conv3D(32, kernel_size=(3,3,3), strides=(1,1,1) , input_shape=input_shape,padding='same'))
model7.add(BatchNormalization())
model7.add(Activation('relu'))
model7.add(MaxPooling3D(pool_size=(2,2,2)))


# 2nd Layer Group
model7.add(Conv3D(64, kernel_size=(3,3,3),strides=(1,1,1) ,padding='same'))
model7.add(BatchNormalization())
model7.add(Activation('relu'))
model7.add(MaxPooling3D(pool_size=(2,2,2)))

# 3rd Layer Group
model7.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model7.add(BatchNormalization())
model7.add(Activation('relu'))
model7.add(MaxPooling3D(pool_size=(2,2,2)))


#FC layer group
model7.add(Flatten())
model7.add(Dense(512,activation='relu'))
model7.add(Dropout(0.5))

model7.add(Dense(128,activation='relu'))
model7.add(Dropout(0.25))

# Softmax Layer
model7.add(Dense(5, activation='softmax'))

In [40]:
optimiser = Adam() #write your optimizer
model7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model7.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 15, 80, 80, 32)    2624      
_________________________________________________________________
batch_normalization_19 (Batc (None, 15, 80, 80, 32)    128       
_________________________________________________________________
activation_19 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 7, 40, 40, 32)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 7, 40, 40, 64)     55360     
_________________________________________________________________
batch_normalization_20 (Batc (None, 7, 40, 40, 64)     256       
_________________________________________________________________
activation_20 (Activation)   (None, 7, 40, 40, 64)     0         
__________

# Conv2D+RNN (LSTM) Models

 ## Model 8

In [41]:
Input_shape = (15,80,80,3)

In [42]:
# Model8
model8 = Sequential()

#1st Conv2D group
model8.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(MaxPooling2D((2, 2))))

#2nd Conv2D group
model8.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(MaxPooling2D((2, 2))))

#3rd Conv2D group
model8.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(TimeDistributed(MaxPooling2D((2, 2))))

#FC layer group
model8.add(TimeDistributed(Flatten()))
#LSTM
model8.add(LSTM(128, return_sequences=False, dropout=0.5))
model8.add(Dense(64,activation='relu'))
model8.add(Dropout(0.5))

#Softmax layer
model8.add(Dense(5, activation='softmax'))

In [43]:
optimiser = Adam() #write your optimizer
model8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model8.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 80, 80, 32)    896       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 80, 80, 32)    128       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 80, 80, 32)    9248      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 80, 80, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 40, 40, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 40, 40, 64)    18496     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 15, 40, 40, 64)    256       
__________

 ## Model 9

In [44]:
# Model9
model9 = Sequential()

#1st Conv2D group
model9.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model9.add(TimeDistributed(BatchNormalization()))
model9.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model9.add(TimeDistributed(BatchNormalization()))
model9.add(TimeDistributed(MaxPooling2D((2, 2))))

#2nd Conv2D group
model9.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model9.add(TimeDistributed(BatchNormalization()))
model9.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model9.add(TimeDistributed(BatchNormalization()))
model9.add(TimeDistributed(MaxPooling2D((2, 2))))

#3rd Conv2D group
model9.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model9.add(TimeDistributed(BatchNormalization()))
model9.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model9.add(TimeDistributed(BatchNormalization()))
model9.add(TimeDistributed(MaxPooling2D((2, 2))))

#FC layer group
model9.add(TimeDistributed(Flatten()))
#LSTM
model9.add(LSTM(128, return_sequences=False, dropout=0.5))
model9.add(Dense(64,activation='relu'))
model9.add(Dropout(0.5))

#Softmax Layer
model9.add(Dense(5, activation='softmax'))

In [45]:
optimiser = Adam() #write your optimizer
model9.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model9.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_17 (TimeDis (None, 15, 80, 80, 16)    448       
_________________________________________________________________
time_distributed_18 (TimeDis (None, 15, 80, 80, 16)    64        
_________________________________________________________________
time_distributed_19 (TimeDis (None, 15, 80, 80, 16)    2320      
_________________________________________________________________
time_distributed_20 (TimeDis (None, 15, 80, 80, 16)    64        
_________________________________________________________________
time_distributed_21 (TimeDis (None, 15, 40, 40, 16)    0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 15, 40, 40, 32)    4640      
_________________________________________________________________
time_distributed_23 (TimeDis (None, 15, 40, 40, 32)    128       
__________

# Conv2D + RNN (GRU) Models

 ## Model 10

In [46]:
Input_shape = (15,80,80,3)

In [47]:
# Model10
model10 = Sequential()

#1st Conv2D group
model10.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(MaxPooling2D((2, 2))))

#2nd Conv2D group
model10.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(MaxPooling2D((2, 2))))

#3rd Conv2D group
model10.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(TimeDistributed(MaxPooling2D((2, 2))))

#FC layer group
model10.add(TimeDistributed(Flatten()))
#GRU
model10.add(GRU(128, return_sequences=False, dropout=0.5))
model10.add(Dense(64,activation='relu'))
model10.add(Dropout(0.5))

#Softmax layer
model10.add(Dense(5, activation='softmax'))

In [48]:
optimiser = Adam() #write your optimizer
model10.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model10.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_33 (TimeDis (None, 15, 80, 80, 32)    896       
_________________________________________________________________
time_distributed_34 (TimeDis (None, 15, 80, 80, 32)    128       
_________________________________________________________________
time_distributed_35 (TimeDis (None, 15, 80, 80, 32)    9248      
_________________________________________________________________
time_distributed_36 (TimeDis (None, 15, 80, 80, 32)    128       
_________________________________________________________________
time_distributed_37 (TimeDis (None, 15, 40, 40, 32)    0         
_________________________________________________________________
time_distributed_38 (TimeDis (None, 15, 40, 40, 64)    18496     
_________________________________________________________________
time_distributed_39 (TimeDis (None, 15, 40, 40, 64)    256       
__________

 ## Model 11

In [49]:
# Model11
model11 = Sequential()

#1st Conv2D group
model11.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model11.add(TimeDistributed(BatchNormalization()))
model11.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model11.add(TimeDistributed(BatchNormalization()))
model11.add(TimeDistributed(MaxPooling2D((2, 2))))

#2nd Conv2D group
model11.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model11.add(TimeDistributed(BatchNormalization()))
model11.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model11.add(TimeDistributed(BatchNormalization()))
model11.add(TimeDistributed(MaxPooling2D((2, 2))))

#3rd Conv2D group
model11.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu'), input_shape=Input_shape))
model11.add(TimeDistributed(BatchNormalization()))
model11.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model11.add(TimeDistributed(BatchNormalization()))
model11.add(TimeDistributed(MaxPooling2D((2, 2))))

#FC layer group
model11.add(TimeDistributed(Flatten()))
#GRU
model11.add(GRU(128, return_sequences=False, dropout=0.5))
model11.add(Dense(64,activation='relu'))
model11.add(Dropout(0.5))

#Softmax Layer
model11.add(Dense(5, activation='softmax'))

In [50]:
optimiser = Adam() #write your optimizer
model11.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model11.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_49 (TimeDis (None, 15, 80, 80, 16)    448       
_________________________________________________________________
time_distributed_50 (TimeDis (None, 15, 80, 80, 16)    64        
_________________________________________________________________
time_distributed_51 (TimeDis (None, 15, 80, 80, 16)    2320      
_________________________________________________________________
time_distributed_52 (TimeDis (None, 15, 80, 80, 16)    64        
_________________________________________________________________
time_distributed_53 (TimeDis (None, 15, 40, 40, 16)    0         
_________________________________________________________________
time_distributed_54 (TimeDis (None, 15, 40, 40, 32)    4640      
_________________________________________________________________
time_distributed_55 (TimeDis (None, 15, 40, 40, 32)    128       
__________

# Transfer Learning Models

### Non-Trainable Layers

In [51]:
input_shape= (15,80,80,3)

## Model 12

In [52]:
# Model12: Transfer Learning with LSTM :-

from keras.applications import mobilenet
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

model12 = Sequential()
model12.add(TimeDistributed(mobilenet_transfer, input_shape=input_shape))
        
for layer in model12.layers:
    layer.trainable = False
        
model12.add(TimeDistributed(BatchNormalization()))
model12.add(TimeDistributed(MaxPooling2D((2, 2))))

model12.add(TimeDistributed(Flatten()))
model12.add(LSTM(128,return_sequences=False, dropout=0.5))
model12.add(Dense(64,activation='relu'))
model12.add(Dropout(0.5))

#Softmax Layer
model12.add(Dense(5, activation='softmax'))

17227776/17225924 [==============================] - 2s 0us/step


In [53]:
optimiser = Adam() #write your optimizer
model12.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model12.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_65 (TimeDis (None, 15, 2, 2, 1024)    3228864   
_________________________________________________________________
time_distributed_66 (TimeDis (None, 15, 2, 2, 1024)    4096      
_________________________________________________________________
time_distributed_67 (TimeDis (None, 15, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_68 (TimeDis (None, 15, 1024)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dense_26 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
__________

## Model 13

In [54]:
# Model13: Transfer Learning with GRU :-
from keras.applications import mobilenet
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

model13 = Sequential()
model13.add(TimeDistributed(mobilenet_transfer, input_shape=input_shape))
        
for layer in model13.layers:
    layer.trainable = False
        
model13.add(TimeDistributed(BatchNormalization()))
model13.add(TimeDistributed(MaxPooling2D((2, 2))))

model13.add(TimeDistributed(Flatten()))

#model13.add(Dropout(0.5))
model13.add(GRU(128,return_sequences=False, dropout=0.5))
model13.add(Dense(64,activation='relu'))
model13.add(Dropout(0.5))
model13.add(Dense(5, activation='softmax'))

In [55]:
optimiser = Adam() #write your optimizer
model13.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model13.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_69 (TimeDis (None, 15, 2, 2, 1024)    3228864   
_________________________________________________________________
time_distributed_70 (TimeDis (None, 15, 2, 2, 1024)    4096      
_________________________________________________________________
time_distributed_71 (TimeDis (None, 15, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_72 (TimeDis (None, 15, 1024)          0         
_________________________________________________________________
gru_3 (GRU)                  (None, 128)               442752    
_________________________________________________________________
dense_28 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_16 (Dropout)         (None, 64)                0         
__________

### Trainable Layers

In [56]:
input_shape= (15,80,80,3)

## Model 14

In [57]:
# Model14: Transfer Learning with LSTM :-

from keras.applications import mobilenet
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

model14 = Sequential()
model14.add(TimeDistributed(mobilenet_transfer, input_shape=input_shape))
             
model14.add(TimeDistributed(BatchNormalization()))
model14.add(TimeDistributed(MaxPooling2D((2, 2))))

model14.add(TimeDistributed(Flatten()))
model14.add(LSTM(128,return_sequences=False, dropout=0.5))
model14.add(Dense(64,activation='relu'))
model14.add(Dropout(0.5))

#Softmax Layer
model14.add(Dense(5, activation='softmax'))

In [58]:
optimiser = Adam() #write your optimizer
model14.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model14.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_73 (TimeDis (None, 15, 2, 2, 1024)    3228864   
_________________________________________________________________
time_distributed_74 (TimeDis (None, 15, 2, 2, 1024)    4096      
_________________________________________________________________
time_distributed_75 (TimeDis (None, 15, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_76 (TimeDis (None, 15, 1024)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dense_30 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
__________

## Model 15

In [59]:
# Model15: Transfer Learning with GRU :-

from keras.applications import mobilenet
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

model15 = Sequential()
model15.add(TimeDistributed(mobilenet_transfer, input_shape=input_shape))
        
model15.add(TimeDistributed(BatchNormalization()))
model15.add(TimeDistributed(MaxPooling2D((2, 2))))

model15.add(TimeDistributed(Flatten()))

#model15.add(Dropout(0.5))
model15.add(GRU(128,return_sequences=False, dropout=0.5))
model15.add(Dense(64,activation='relu'))
model15.add(Dropout(0.5))
model15.add(Dense(5, activation='softmax'))

In [60]:
optimiser = Adam() #write your optimizer
model15.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model15.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_77 (TimeDis (None, 15, 2, 2, 1024)    3228864   
_________________________________________________________________
time_distributed_78 (TimeDis (None, 15, 2, 2, 1024)    4096      
_________________________________________________________________
time_distributed_79 (TimeDis (None, 15, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_80 (TimeDis (None, 15, 1024)          0         
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               442752    
_________________________________________________________________
dense_32 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
__________

In [61]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [62]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [63]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [31]:
history1= model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = Source path =  ./Project_data/train ; batch size = 32
32
Epoch 1/30
21/21 [==============================] - 92s 4s/step - loss: 3.8068 - categorical_accuracy: 0.2909 - val_loss: 1.9448 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-12-2711_18_53.639796/model-00001-3.84132-0.28959-1.94481-0.25000.h5
Epoch 2/30
21/21 [==============================] - 18s 847ms/step - loss: 1.3026 - categorical_accuracy: 0.4410 - val_loss: 1.5686 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_init_2020-12-2711_18_53.639796/model-00002-1.30258-0.44099-1.56859-0.18750.h5
Epoch 3/30
21/21 [==============================] - 15s 730ms/step - loss: 0.9688 - categorical_accuracy: 0.6093 - val_loss: 1.2303 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-12-2711_18_53.639796/model-00003-0.97714-0.60656-1.23029-0.50000.h5
Epoch 4/30
21/21 [==============================] - 14

In [33]:
history2= model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 12s 578ms/step - loss: 7.6726 - categorical_accuracy: 0.2717 - val_loss: 11.7609 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-12-2711_18_53.639796/model-00001-7.67262-0.27171-11.76090-0.25000.h5
Epoch 2/30
21/21 [==============================] - 11s 511ms/step - loss: 4.3527 - categorical_accuracy: 0.2969 - val_loss: 1.4953 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_init_2020-12-2711_18_53.639796/model-00002-4.35272-0.29692-1.49534-0.18750.h5
Epoch 3/30
21/21 [==============================] - 11s 542ms/step - loss: 1.3281 - categorical_accuracy: 0.4090 - val_loss: 1.3895 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-12-2711_18_53.639796/model-00003-1.32813-0.40896-1.38948-0.50000.h5
Epoch 4/30
21/21 [==============================] - 11s 522ms/step - loss: 1.2102 - categorical_accuracy: 0.4762 - val_loss: 0.7866 - val_categorical_accur

In [35]:
history3= model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 12s 584ms/step - loss: 11.1055 - categorical_accuracy: 0.2269 - val_loss: 11.7389 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-12-2711_18_53.639796/model-00001-11.10555-0.22689-11.73892-0.25000.h5
Epoch 2/30
21/21 [==============================] - 11s 523ms/step - loss: 12.1274 - categorical_accuracy: 0.2129 - val_loss: 13.0960 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_init_2020-12-2711_18_53.639796/model-00002-12.12738-0.21289-13.09595-0.18750.h5
Epoch 3/30
21/21 [==============================] - 11s 526ms/step - loss: 11.2773 - categorical_accuracy: 0.2577 - val_loss: 8.3435 - val_categorical_accuracy: 0.3125

Epoch 00003: saving model to model_init_2020-12-2711_18_53.639796/model-00003-11.27725-0.25770-8.34349-0.31250.h5
Epoch 4/30
21/21 [==============================] - 11s 523ms/step - loss: 10.6700 - categorical_accuracy: 0.2857 - val_loss: 12.0683 - val_categor

In [45]:
history4= model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 13s 624ms/step - loss: 9.2715 - categorical_accuracy: 0.2633 - val_loss: 11.0812 - val_categorical_accuracy: 0.3125

Epoch 00001: saving model to model_init_2020-12-2711_18_53.639796/model-00001-9.27151-0.26331-11.08119-0.31250.h5
Epoch 2/30
21/21 [==============================] - 10s 496ms/step - loss: 9.1892 - categorical_accuracy: 0.2605 - val_loss: 5.8599 - val_categorical_accuracy: 0.3750

Epoch 00002: saving model to model_init_2020-12-2711_18_53.639796/model-00002-9.18923-0.26050-5.85987-0.37500.h5
Epoch 3/30
21/21 [==============================] - 11s 506ms/step - loss: 5.6181 - categorical_accuracy: 0.3081 - val_loss: 8.6797 - val_categorical_accuracy: 0.3125

Epoch 00003: saving model to model_init_2020-12-2711_18_53.639796/model-00003-5.61809-0.30812-8.67968-0.31250.h5
Epoch 4/30
21/21 [==============================] - 11s 539ms/step - loss: 1.3427 - categorical_accuracy: 0.4314 - val_loss: 5.7045 - val_categorical_accur

In [40]:
history5= model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size =Epoch 1/30
 32
21/21 [==============================] - 23s 1s/step - loss: 2.4084 - categorical_accuracy: 0.2937 - val_loss: 1.5069 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-12-2711_40_24.647928/model-00001-2.41729-0.29412-1.50690-0.25000.h5
Epoch 2/30
21/21 [==============================] - 15s 707ms/step - loss: 1.5415 - categorical_accuracy: 0.3685 - val_loss: 1.1205 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-12-2711_40_24.647928/model-00002-1.54149-0.36853-1.12053-0.50000.h5
Epoch 3/30
21/21 [==============================] - 13s 617ms/step - loss: 1.3041 - categorical_accuracy: 0.4223 - val_loss: 1.0976 - val_categorical_accuracy: 0.6875

Epoch 00003: saving model to model_init_2020-12-2711_40_24.647928/model-00003-1.30638-0.42389-1.09756-0.68750.h5
Epoch 4/30
21/21 [==============================] - 13

In [42]:
history6= model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 12s 571ms/step - loss: 2.6207 - categorical_accuracy: 0.2129 - val_loss: 3.8426 - val_categorical_accuracy: 0.1875

Epoch 00001: saving model to model_init_2020-12-2711_40_24.647928/model-00001-2.62074-0.21289-3.84255-0.18750.h5
Epoch 2/30
21/21 [==============================] - 10s 484ms/step - loss: 1.5513 - categorical_accuracy: 0.2997 - val_loss: 3.1461 - val_categorical_accuracy: 0.3750

Epoch 00002: saving model to model_init_2020-12-2711_40_24.647928/model-00002-1.55129-0.29972-3.14606-0.37500.h5
Epoch 3/30
21/21 [==============================] - 12s 562ms/step - loss: 1.4136 - categorical_accuracy: 0.3754 - val_loss: 2.5756 - val_categorical_accuracy: 0.1250

Epoch 00003: saving model to model_init_2020-12-2711_40_24.647928/model-00003-1.41356-0.37535-2.57563-0.12500.h5
Epoch 4/30
21/21 [==============================] - 11s 531ms/step - loss: 1.4012 - categorical_accuracy: 0.3754 - val_loss: 3.3840 - val_categorical_accurac

In [48]:
history7= model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 14s 649ms/step - loss: 7.1450 - categorical_accuracy: 0.2381 - val_loss: 3.1092 - val_categorical_accuracy: 0.3750

Epoch 00001: saving model to model_init_2020-12-2711_40_24.647928/model-00001-7.14503-0.23810-3.10924-0.37500.h5
Epoch 2/30
21/21 [==============================] - 9s 448ms/step - loss: 2.1482 - categorical_accuracy: 0.2493 - val_loss: 1.7200 - val_categorical_accuracy: 0.3125

Epoch 00002: saving model to model_init_2020-12-2711_40_24.647928/model-00002-2.14819-0.24930-1.72002-0.31250.h5
Epoch 3/30
21/21 [==============================] - 12s 558ms/step - loss: 1.3425 - categorical_accuracy: 0.4118 - val_loss: 1.5124 - val_categorical_accuracy: 0.3125

Epoch 00003: saving model to model_init_2020-12-2711_40_24.647928/model-00003-1.34245-0.41176-1.51238-0.31250.h5
Epoch 4/30
21/21 [==============================] - 12s 551ms/step - loss: 1.2750 - categorical_accuracy: 0.4202 - val_loss: 2.1827 - val_categorical_accuracy

In [54]:
history8= model8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size =Epoch 1/30
 32
21/21 [==============================] - 25s 1s/step - loss: 1.5872 - categorical_accuracy: 0.2740 - val_loss: 1.6007 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-12-2713_52_38.997171/model-00001-1.58817-0.27300-1.60070-0.22000.h5
Epoch 2/30
21/21 [==============================] - 13s 627ms/step - loss: 1.3877 - categorical_accuracy: 0.4244 - val_loss: 1.7744 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_init_2020-12-2713_52_38.997171/model-00002-1.38772-0.42443-1.77445-0.18750.h5
Epoch 3/30
21/21 [==============================] - 12s 565ms/step - loss: 1.3029 - categorical_accuracy: 0.4686 - val_loss: 1.9531 - val_categorical_accuracy: 0.1250

Epoch 00003: saving model to model_init_2020-12-2713_52_38.997171/model-00003-1.30765-0.46604-1.95312-0.12500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate

In [38]:
history9= model9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 14s 674ms/step - loss: 1.6501 - categorical_accuracy: 0.2409 - val_loss: 1.7179 - val_categorical_accuracy: 0.1875

Epoch 00001: saving model to model_init_2020-12-2713_37_00.524791/model-00001-1.65014-0.24090-1.71790-0.18750.h5
Epoch 2/30
21/21 [==============================] - 9s 420ms/step - loss: 1.4175 - categorical_accuracy: 0.3866 - val_loss: 1.3279 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-12-2713_37_00.524791/model-00002-1.41746-0.38655-1.32787-0.50000.h5
Epoch 3/30
21/21 [==============================] - 11s 522ms/step - loss: 1.3230 - categorical_accuracy: 0.4426 - val_loss: 1.1261 - val_categorical_accuracy: 0.6875

Epoch 00003: saving model to model_init_2020-12-2713_37_00.524791/model-00003-1.32297-0.44258-1.12610-0.68750.h5
Epoch 4/30
21/21 [==============================] - 12s 559ms/step - loss: 1.2258 - categorical_accuracy: 0.4846 - val_loss: 1.1147 - val_categorical_accuracy

In [17]:
history10= model10.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/trainEpoch 1/30
 ; batch size = 32
21/21 [==============================] - 31s 1s/step - loss: 1.6974 - categorical_accuracy: 0.2397 - val_loss: 1.4495 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_2020-12-2714_18_00.104298/model-00001-1.70086-0.23831-1.44947-0.39000.h5
Epoch 2/30
21/21 [==============================] - 12s 584ms/step - loss: 1.3705 - categorical_accuracy: 0.4555 - val_loss: 1.5008 - val_categorical_accuracy: 0.4375

Epoch 00002: saving model to model_init_2020-12-2714_18_00.104298/model-00002-1.37049-0.45549-1.50084-0.43750.h5
Epoch 3/30
21/21 [==============================] - 13s 634ms/step - loss: 1.2873 - categorical_accuracy: 0.4794 - val_loss: 1.5801 - val_categorical_accuracy: 0.4375

Epoch 00003: saving model to model_init_2020-12-2714_18_00.104298/model-00003-1.29073-0.47775-1.58008-0.43750.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate

In [19]:
history11= model11.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 12s 580ms/step - loss: 1.7037 - categorical_accuracy: 0.3025 - val_loss: 1.4066 - val_categorical_accuracy: 0.4375

Epoch 00001: saving model to model_init_2020-12-2714_18_00.104298/model-00001-1.70372-0.30252-1.40665-0.43750.h5
Epoch 2/30
21/21 [==============================] - 10s 498ms/step - loss: 1.4660 - categorical_accuracy: 0.3754 - val_loss: 1.6307 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-12-2714_18_00.104298/model-00002-1.46597-0.37535-1.63074-0.25000.h5
Epoch 3/30
21/21 [==============================] - 11s 539ms/step - loss: 1.3914 - categorical_accuracy: 0.3950 - val_loss: 1.1860 - val_categorical_accuracy: 0.5625

Epoch 00003: saving model to model_init_2020-12-2714_18_00.104298/model-00003-1.39141-0.39496-1.18595-0.56250.h5
Epoch 4/30
21/21 [==============================] - 11s 528ms/step - loss: 1.2671 - categorical_accuracy: 0.4314 - val_loss: 1.7259 - val_categorical_accurac

In [17]:
history12= model12.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size = 32Epoch 1/30

21/21 [==============================] - 26s 1s/step - loss: 1.6475 - categorical_accuracy: 0.2446 - val_loss: 1.6330 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2020-12-2715_29_07.909081/model-00001-1.64979-0.24434-1.63301-0.24000.h5
Epoch 2/30
21/21 [==============================] - 13s 601ms/step - loss: 1.5070 - categorical_accuracy: 0.3499 - val_loss: 1.4850 - val_categorical_accuracy: 0.4375

Epoch 00002: saving model to model_init_2020-12-2715_29_07.909081/model-00002-1.50700-0.34990-1.48504-0.43750.h5
Epoch 3/30
21/21 [==============================] - 13s 638ms/step - loss: 1.4351 - categorical_accuracy: 0.3878 - val_loss: 1.9895 - val_categorical_accuracy: 0.0625

Epoch 00003: saving model to model_init_2020-12-2715_29_07.909081/model-00003-1.43833-0.38642-1.98947-0.06250.h5
Epoch 4/30
21/21 [==============================] - 12

In [19]:
history13= model13.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
21/21 [==============================] - 13s 621ms/step - loss: 1.8672 - categorical_accuracy: 0.2661 - val_loss: 1.4052 - val_categorical_accuracy: 0.3125

Epoch 00001: saving model to model_init_2020-12-2715_29_07.909081/model-00001-1.86716-0.26611-1.40522-0.31250.h5
Epoch 2/30
21/21 [==============================] - 11s 502ms/step - loss: 1.6232 - categorical_accuracy: 0.3137 - val_loss: 1.4021 - val_categorical_accuracy: 0.3125

Epoch 00002: saving model to model_init_2020-12-2715_29_07.909081/model-00002-1.62315-0.31373-1.40205-0.31250.h5
Epoch 3/30
21/21 [==============================] - 12s 560ms/step - loss: 1.4973 - categorical_accuracy: 0.3501 - val_loss: 1.5140 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-12-2715_29_07.909081/model-00003-1.49733-0.35014-1.51397-0.25000.h5
Epoch 4/30
21/21 [==============================] - 12s 563ms/step - loss: 1.4902 - categorical_accuracy: 0.3613 - val_loss: 1.7696 - val_categorical_accurac

In [17]:
history14= model14.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path = Epoch 1/30
 ./Project_data/train ; batch size = 32
21/21 [==============================] - 100s 5s/step - loss: 1.5989 - categorical_accuracy: 0.3069 - val_loss: 1.3752 - val_categorical_accuracy: 0.4700

Epoch 00001: saving model to model_init_2020-12-2715_24_58.874130/model-00001-1.60254-0.30468-1.37515-0.47000.h5
Epoch 2/30
21/21 [==============================] - 16s 745ms/step - loss: 1.2572 - categorical_accuracy: 0.4948 - val_loss: 0.9024 - val_categorical_accuracy: 0.5625

Epoch 00002: saving model to model_init_2020-12-2715_24_58.874130/model-00002-1.25717-0.49482-0.90238-0.56250.h5
Epoch 3/30
21/21 [==============================] - 17s 826ms/step - loss: 0.9216 - categorical_accuracy: 0.6439 - val_loss: 0.8476 - val_categorical_accuracy: 0.6875

Epoch 00003: saving model to model_init_2020-12-2715_24_58.874130/model-00003-0.92622-0.64403-0.84760-0.68750.h5
Epoch 4/30
21/21 [==============================] - 1

In [37]:
history15= model15.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - 30s 1s/step - loss: 1.7309 - categorical_accuracy: 0.2870 - val_loss: 1.3849 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-12-2715_50_52.109010/model-00001-1.73513-0.28507-1.38494-0.41000.h5
Epoch 2/30
21/21 [==============================] - 12s 593ms/step - loss: 1.1822 - categorical_accuracy: 0.5259 - val_loss: 1.0706 - val_categorical_accuracy: 0.5625

Epoch 00002: saving model to model_init_2020-12-2715_50_52.109010/model-00002-1.18225-0.52588-1.07058-0.56250.h5
Epoch 3/30
21/21 [==============================] - 12s 556ms/step - loss: 0.8409 - categorical_accuracy: 0.6697 - val_loss: 1.0148 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-12-2715_50_52.109010/model-00003-0.85009-0.66745-1.01482-0.50000.h5
Epoch 4/30
21/21 [==============================] - 13